In [1]:
import os
import sys
import tqdm
import pandas as pd

In [2]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [3]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

2


In [4]:
input_dir = 'outputs_4_calculate_properties'

modelnames = [
    'zinc15',
]

filenames = [f'smi_after.csv.{num:04d}' for num in range(50,550,50)]

In [5]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [6]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

100%|██████████| 10/10 [00:07<00:00,  1.26it/s]


In [7]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

,SR,Nov,Div,Model,Ckpt,HMean
0,0.0008,1.0,0.911827,zinc15,0050,0.090236
1,0.0032,1.0,0.761832,zinc15,0100,0.134857
2,0.0020,1.0,0.590627,zinc15,0150,0.105947
3,0.0550,1.0,0.468041,zinc15,0200,0.295628
4,0.4674,1.0,0.474016,zinc15,0250,0.605404
5,0.8266,1.0,0.449494,zinc15,0300,0.719145
6,0.9726,1.0,0.455653,zinc15,0350,0.762619
7,0.9860,1.0,0.481552,zinc15,0400,0.780335
8,0.9854,1.0,0.511271,zinc15,0450,0.795891
9,0.9868,1.0,0.527039,zinc15,0500,0.804363
